RAG 실습

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import getpass
import os

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

rag tutorial

In [53]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/opt/homebrew/anaconda3/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [6]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import JsonOutputParser
llm = ChatOpenAI(model="gpt-4o-mini")

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(["https://namu.wiki/w/%EC%86%8C%EC%84%9C%EB%A6%AC%EC%8A%A4(%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC)","https://namu.wiki/w/%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC","https://www.inven.co.kr/board/lostark/5770/77602","https://namu.wiki/w/%EC%86%8C%EC%84%9C%EB%A6%AC%EC%8A%A4(%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC)/%EC%8A%A4%ED%82%AC"]
)
docs = loader.load()

skill_loader = WebBaseLoader(["https://namu.wiki/w/%EC%86%8C%EC%84%9C%EB%A6%AC%EC%8A%A4(%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC)/%EC%8A%A4%ED%82%AC","https://namu.wiki/w/%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC/%EC%84%B1%EC%9E%A5#s-3.3"])
skill_docs = skill_loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

skill_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
skill_splits = text_splitter.split_documents(docs)

skill_vectorstore = Chroma.from_documents(documents=skill_splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
skill_retriever = skill_vectorstore.as_retriever()

system_prompt = (
"""
    context:  {context} 
    이건 로스트아크의 스킬 및 시스템 정보야 {skill}
    정상적인 데이터를 찾았다면 답변해주세요
    3줄 요약해주세요.
    사용자의 요청은 이것입니다. 질문 : {question}

     """
)
parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

hallucination_check_prompt = (
     """
         context는 {context} 는 앞서 chat gpt 가 응답했던 내용이야, 질문인 question: {question} 에 알맞은 대답이라면 질문에 답변해주고,
         그렇지 않으면 No. 를 출력해줘
     """
)


hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", hallucination_check_prompt),
        ("human", "{question}"),
    ]
)


def format_docs(docs):
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

def print_docs(docs):
    print(docs)
    return docs

my_question = "로스트아크 점화 소서리스의 스킬과 룬에 대해 알려줘"
def first_question(docs):
    #print(docs)
    return my_question

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough() ,  "skill" : skill_retriever | format_docs}
    | prompt
    | print_docs
    | llm
    | print_docs
    | StrOutputParser()
    | {"context" : RunnablePassthrough(), "question": first_question }
    | print_docs
    | hallucination_prompt
    | llm
    | StrOutputParser()
)



print(rag_chain.invoke(my_question))

[Document(metadata={'language': 'ko', 'source': 'https://namu.wiki/w/%EC%86%8C%EC%84%9C%EB%A6%AC%EC%8A%A4(%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC)/%EC%8A%A4%ED%82%AC', 'title': '소서리스(로스트아크)/스킬 - 나무위키'}, page_content='소서리스(로스트아크)/스킬 - 나무위키'), Document(metadata={'language': 'ko', 'source': 'https://namu.wiki/w/%EC%86%8C%EC%84%9C%EB%A6%AC%EC%8A%A4(%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC)/%EC%8A%A4%ED%82%AC', 'title': '소서리스(로스트아크)/스킬 - 나무위키'}, page_content='소서리스(로스트아크)/스킬 - 나무위키'), Document(metadata={'language': 'ko', 'source': 'https://namu.wiki/w/%EC%86%8C%EC%84%9C%EB%A6%AC%EC%8A%A4(%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC)/%EC%8A%A4%ED%82%AC', 'title': '소서리스(로스트아크)/스킬 - 나무위키'}, page_content='소서리스(로스트아크)/스킬 - 나무위키'), Document(metadata={'language': 'ko', 'source': 'https://namu.wiki/w/%EC%86%8C%EC%84%9C%EB%A6%AC%EC%8A%A4(%EB%A1%9C%EC%8A%A4%ED%8A%B8%EC%95%84%ED%81%AC)/%EC%8A%A4%ED%82%AC', 'title': '소서리스(로스트아크)/스킬 - 나무위키'}, page_content='소서리스(로스트아크)/스킬 - 나무위키')]
[Document(m

langchain 을 활용하여 주번 생성 할루시네이션 제거

In [40]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import JsonOutputParser
llm = ChatOpenAI(model="gpt-4o-mini")
system_prompt = (
"""
**프롬프트: 주번 페어 배정 시스템**

주번 페어를 구성하는 시스템을 만드세요. 아래 조건을 반드시 준수하세요:

1. 주번은 매주 월요일에 변경되며, 각 주에 정 1명, 부 1명으로 구성됩니다.
2. 참여자는 다음과 같습니다: 
   - 정: albus.star, dante.93, kylie.do, rein.withus, seed.sim, pharrell.mp3 
   - 부: sera.son, jett.jistol, mark.dd, daisy.potter2, moka.gold, muscle.kim
3. 정과 정은 같은 페어에 포함될 수 없으며, 부와 부도 마찬가지입니다. 예를 들어 albus.star 와 dante.93 은 같이 배정이 안됩니다.
4. 각 사이클에서는 정과 부가 한 번씩 배치되어야 하며, 모든 정/부는 반드시 한 사이클 내에 포함되어야 합니다.
5. cooper.lee는 예비 인원으로, 다른 사람과 페어를 만들 수 없는 경우에만 사용될 수 있습니다.
6. 만약 어느 주에 휴가자가 있다면, 해당 주에는 주번 페어를 휴가자가 아닌 다른 사람으로 배정해야 합니다. 예를 들어, albus.star가 2주차에 휴가인 경우, albus.star 는 다른 주차에 반드시 배정되어야 합니다.
7. 주번 배정 결과를 출력하며, 만약 한 사이클에 포함되지 않은 사람이 있다면 그 사실을 알려주세요.
8. 각 정과 각 부는 무조건 한사이클에 최소 한번은 포함되어야합니다.    
9. 같은 유닛은 같은 주에 한명만 배정받아야합니다. 
- 마들렌 유닛에 속한 사람들: (albus.star, kylie.do, sera.son, jett.jistol)
- 마카롱 유닛에 속한 사람들: (dante.93, muscle.kim, moka.gold, daisy.potter2)
- 타르트 유닛에 속한 사람들: (seed.sim, rein.withus, mark.dd, pharrell.mp3)

아래는 하나의 예시입니다.
----
albus.star 가 1주차에 휴가이고, kylie.do 가 2주차에 휴가입니다.
albus.star 는 1주차에 휴가이기 때문에 1주차에 배정이 안됩니다.
kylie.do 는 2주차에 휴가이기 때문에 2주차에 배정이 안됩니다.

그럴 때 만들 수 경우의 수 중 하나는 아래와 같습니다.
1주차 kylie.do (정-마들렌 유닛), moka.gold (부-마카롱 유닛)
2주차 albus.star (정-마들렌 유닛), mark.dd (부-타르트 유닛)
3주차 pharrell.mp3 (정-타르트 유닛), muscle.kim (부-마카롱 유닛)
4주차 seed.sim (정-타르트 유닛), daisy.potter2 (부-마카롱 유닛)
5주차 rein.withus (정-타르트 유닛), sera.son(부-마들렌 유닛)
6주차 dante.93 (정-마카롱 유닛), jett.jistol(부-마들렌 유닛)

유닛이 겹치는 주번 페어가 하나도 없고, 최소 한명은 주번을 수행하고 있고, 두번이상하지 않았으므로 매우 정상적입니다.

아래와 같은 경우는 불가능합니다.
1주차 kylie.do (정-마들렌 유닛), albus.star (부-마들렌 유닛)
2주차 albus.star (정-마들렌 유닛), mark.dd (부-타르트 유닛)
3주차 pharrell.mp3 (정-타르트 유닛), muscle.kim (부-마카롱 유닛)
4주차 mark.dd (정-타르트 유닛), daisy.potter2 (부-마카롱 유닛)
5주차 rein.withus (정-타르트 유닛), sera.son(부-마들렌 유닛)
6주차 albus.star (정-마카롱 유닛), jett.jistol(부-마들렌 유닛)

albus.star 는 정 역할입니다. 또한 kylie.do 도 정 역할이기 때문에 1주차에 같이 배정될 수 없습니다.
mark.dd 는 정 역할이 아닌 부 역할입니다. 따라서 4주차의 배정은 잘못되었습니다.
albus.star 는 두 번 이상 배정되었습니다. 따라서 배정이 잘못되었습니다.
dante.93 은 목록에 빠져있습니다. 따라서 배정이 잘못되었습니다.
albus.star 과 jett.jisotl 은 둘 다 마들렌 유닛이기 때문에,  6주차의 배정은 잘못 되었습니다.

----

위 규칙은 반드시 지켜져야합니다. 위 예시를 참고합니다.
사용자의 요청은 다음과 같습니다.
question : {question}

결과를 테이블과 사용자의 요청을 포함해 출력해주세요.
"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

halluciation_system_prompt = (
"""
**프롬프트: 주번 페어 배정 시스템**

주번 페어를 구성하는 시스템을 만드세요. 아래 조건을 반드시 준수하세요:

1. 주번은 매주 월요일에 변경되며, 각 주에 정 1명, 부 1명으로 구성됩니다.
2. 참여자는 다음과 같습니다: 
   - 정: albus.star, dante.93, kylie.do, rein.withus, seed.sim, pharrell.mp3 
   - 부: sera.son, jett.jistol, mark.dd, daisy.potter2, moka.gold, muscle.kim
3. 정과 정은 같은 페어에 포함될 수 없으며, 부와 부도 마찬가지입니다. 예를 들어 albus.star 와 dante.93 은 같이 배정이 안됩니다.
4. 각 사이클에서는 정과 부가 한 번씩 배치되어야 하며, 모든 정/부는 반드시 한 사이클 내에 포함되어야 합니다.
5. cooper.lee는 예비 인원으로, 다른 사람과 페어를 만들 수 없는 경우에만 사용될 수 있습니다.
6. 만약 어느 주에 휴가자가 있다면, 해당 주에는 주번 페어를 휴가자가 아닌 다른 사람으로 배정해야 합니다. 예를 들어, albus.star가 2주차에 휴가인 경우, albus.star 는 다른 주차에 반드시 배정되어야 합니다.
7. 주번 배정 결과를 출력하며, 만약 한 사이클에 포함되지 않은 사람이 있다면 그 사실을 알려주세요.
8. 각 정과 각 부는 무조건 한사이클에 최소 한번은 포함되어야합니다.    
9. 같은 유닛은 같은 주에 한명만 배정받아야합니다. 
- 마들렌 유닛에 속한 사람들: (albus.star, kylie.do, sera.son, jett.jistol)
- 마카롱 유닛에 속한 사람들: (dante.93, muscle.kim, moka.gold, daisy.potter2)
- 타르트 유닛에 속한 사람들: (seed.sim, rein.withus, mark.dd, pharrell.mp3)

위 규칙들은 반드시 지켜져야합니다. 사전에 만들어진 주번 페어가 충족하는지 단계별로 확인하세요.

아래는 하나의 예시입니다.
----
albus.star 가 1주차에 휴가이고, kylie.do 가 2주차에 휴가입니다.
albus.star 는 1주차에 휴가이기 때문에 1주차에 배정이 안됩니다.
kylie.do 는 2주차에 휴가이기 때문에 2주차에 배정이 안됩니다.

그럴 때 만들 수 경우의 수 중 하나는 아래와 같습니다.
1주차 kylie.do (정-마들렌 유닛), moka.gold (부-마카롱 유닛)
2주차 albus.star (정-마들렌 유닛), mark.dd (부-타르트 유닛)
3주차 pharrell.mp3 (정-타르트 유닛), muscle.kim (부-마카롱 유닛)
4주차 seed.sim (정-타르트 유닛), daisy.potter2 (부-마카롱 유닛)
5주차 rein.withus (정-타르트 유닛), sera.son(부-마들렌 유닛)
6주차 dante.93 (정-마카롱 유닛), jett.jistol(부-마들렌 유닛)

유닛이 겹치는 주번 페어가 하나도 없고, 최소 한명은 주번을 수행하고 있고, 두번이상하지 않았으므로 매우 정상적입니다.

아래와 같은 경우는 불가능합니다.
1주차 kylie.do (정-마들렌 유닛), albus.star (부-마들렌 유닛)
2주차 albus.star (정-마들렌 유닛), mark.dd (부-타르트 유닛)
3주차 pharrell.mp3 (정-타르트 유닛), muscle.kim (부-마카롱 유닛)
4주차 mark.dd (정-타르트 유닛), daisy.potter2 (부-마카롱 유닛)
5주차 rein.withus (정-타르트 유닛), sera.son(부-마들렌 유닛)
6주차 albus.star (정-마카롱 유닛), jett.jistol(부-마들렌 유닛)

albus.star 는 정 역할입니다. 또한 kylie.do 도 정 역할이기 때문에 1주차에 같이 배정될 수 없습니다.
mark.dd 는 정 역할이 아닌 부 역할입니다. 따라서 4주차의 배정은 잘못되었습니다.
albus.star 는 두 번 이상 배정되었습니다. 따라서 배정이 잘못되었습니다.
dante.93 은 목록에 빠져있습니다. 따라서 배정이 잘못되었습니다.
albus.star 과 jett.jisotl 은 둘 다 마들렌 유닛이기 때문에,  6주차의 배정은 잘못 되었습니다.

----

사전에 만들어진 주번 페어는 이렇습니다.
{question}
사전에 만들어진 주번 페어가 주차별로 같은 유닛 사람이 2명이상 배정되었는지도 확인해야합니다.
사전에 만들어진 주번 페어가 위 규칙에 맞는지 확인한 후 검증되었다면 그대로 출력해줍니다.
규칙에 위반되었다면 규칙에 맞게 다시 생성해주세요.
"""
)

halluciation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", halluciation_system_prompt),
        ("human", "{question}"),
    ]
)

rag_chain = (
    {"question" : RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
    |  {"question" : RunnablePassthrough()} 
    | halluciation_prompt
    | llm
    | StrOutputParser()
)

while True:
    result = rag_chain.invoke("albus.star 는 1주차에 휴가이고, jett.jistol 은 5주차에 휴가야. 6주치의 적절한 주번페어를 생성해줘.")
    print(result)
    if result != "No":
        break
    

사전에 만들어진 주번 페어를 확인해 보겠습니다.

| 주차 | 정 (정)                 | 부 (부)                 |
|------|------------------------|------------------------|
| 1주차 | kylie.do (정-마들렌 유닛)    | moka.gold (부-마카롱 유닛)  |
| 2주차 | seed.sim (정-타르트 유닛)    | muscle.kim (부-마카롱 유닛) |
| 3주차 | dante.93 (정-마카롱 유닛)    | mark.dd (부-타르트 유닛)    |
| 4주차 | rein.withus (정-타르트 유닛) | daisy.potter2 (부-마카롱 유닛) |
| 5주차 | pharrell.mp3 (정-타르트 유닛) | sera.son (부-마들렌 유닛)    |
| 6주차 | albus.star (정-마들렌 유닛)   | jett.jistol (부-마들렌 유닛)  |

1. **휴가 확인**:
   - albus.star는 1주차에 휴가이므로 1주차에는 배정되지 않았습니다. 
   - jett.jistol은 5주차에 휴가이므로 5주차에는 배정되지 않았습니다.

2. **정/부 인원 확인**:
   - 모든 정/부는 한 사이클 내에 포함되어 있습니다.
   - 각 유닛에서 같은 주에 배정된 인원은 없습니다:
     - 1주차: kylie.do (마들렌), moka.gold (마카롱)
     - 2주차: seed.sim (타르트), muscle.kim (마카롱)
     - 3주차: dante.93 (마카롱), mark.dd (타르트)
     - 4주차: rein.withus (타르트), daisy.potter2 (마카롱)
     - 5주차: pharrell.mp3 (타르트), sera.son (마들렌)
     - 6주차: albus.star (마들렌), jett.jistol (마들렌)

3. **위반 사항 확인**:
   - 6주차에서 albus.sta